<a href="https://colab.research.google.com/github/Vixuz144/MN1/blob/main/Segundo_examen_MN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
import numpy as np
import sympy as sp
import pandas as pd
import matplotlib.pyplot as plt

In [95]:
x = sp.Symbol('x')

#Primera Pregunta

In [96]:
def Neville(X, Y, x=x, dec=5):
  n = len(X)
  M = np.zeros((n,n+1))*x
  M[:,:2] = np.stack((X,Y),1)
  for i in range(1,n):
    for j in range(n-i):
      M[j,i+1] = ((x-X[j])*M[j+1,i] - (x-X[j+i])*M[j,i]) / (X[j+i]-X[j])
  try:
    M = np.round(M, dec)
    return M
  except:
    return M[0,-1]

In [97]:
X = [0, 0.4, 0.7]
Y = [0, 2.8, 6.4]
print(Neville(X,Y,0.5))
print(Neville(X,Y,0.7))

[[0.      0.      3.5     3.85714]
 [0.4     2.8     4.      0.     ]
 [0.7     6.4     0.      0.     ]]
[[0.  0.  4.9 6.4]
 [0.4 2.8 6.4 0. ]
 [0.7 6.4 0.  0. ]]


#Segunda pregunta

In [98]:
def NDD(X, Y, x = x, dec = 5):
  F = np.ones_like(Y) * Y
  n = len(X)

  M = np.concatenate((np.transpose([X]),np.transpose([Y]),np.zeros((n,n-1))),1)
  p = Y[0] * 1

  for i in range(n-1):
    m = len(F)
    L = []
    for j in range(1,m):
      a = (F[j]- F[j-1])/(X[i+j]-X[j-1])
      L.append(a)
      M[j+i,i+2] = a
    F = np.array(L)

    s = 1
    for j in range(i+1):
      a = x-np.round(X[j], dec)
      s *= a
    p += np.round(F[0], dec)*s

  t = ['X', 'Y']
  for i in range(1,n):
    comas = ','*i
    t.append(f'f({comas})')

  T = pd.DataFrame(np.round(M, dec), columns=t)
  return T, p

In [99]:
X = [0, 0.4, 0.7]
Y = [1, 3, 6]
t,p = NDD(X,Y,dec=5)
print(t)

     X    Y  f(,)    f(,,)
0  0.0  1.0   0.0  0.00000
1  0.4  3.0   5.0  0.00000
2  0.7  6.0  10.0  7.14286


#Tercer pregunta

In [100]:
def inter_lag(X, Y, t = x):
  n = len(X)
  l = []
  for i in range(n):
    a = t-np.append(X[:i], X[i+1:])
    b = X[i]-np.append(X[:i], X[i+1:])
    l.append(np.prod(a/b))
  l = np.array(l)
  return np.dot(Y, l)

In [101]:
X = np.array([0, 0.6, 0.9])
t = 0.45
dec = 7

In [102]:
Ya = np.sin(X)
s1 = np.round(inter_lag(X[:-1],Ya[:-1],t), dec)
print(inter_lag(X[:-1],Ya[:-1]))
print(s1, abs(np.sin(t)-s1))
p = inter_lag(X,Ya).expand()
print(p)
print(inter_lag(X,Ya).expand())
s2 = np.round(inter_lag(X,Ya,t),dec)
print(s2, abs(np.sin(t)-s2),'\n')


q = x*(x-0.6)*(x-0.9)
der = sp.diff(q.expand())
print(der)
ec = sp.Eq(der,0)
sol = sp.solve(ec,x)
print(sol,'\n')

print(q.subs(x,sol[0]), q.subs(x,sol[1]))
print(q.subs(x,sol[0])/6)

0.941070788991726*x
0.4234819 0.011483634111230212
-0.235691853574332*x**2 + 1.08248590113633*x
-0.235691853574332*x**2 + 1.08248590113633*x
0.4393911 0.004425565888769745 

3*x**2 - 3.0*x + 0.54
[0.235424868893541, 0.764575131106459] 

0.0570405183549043 -0.0170405183549043
0.00950675305915071


In [103]:
Yb = np.cos(X)
print(inter_lag(X[:-1],Yb[:-1]))
sol = np.round(inter_lag(X[:-1],Yb[:-1],t),dec)
print(sol, sol - np.cos(t))
print(inter_lag(X,Yb))
print(inter_lag(X,Yb).expand())
sol = np.round(inter_lag(X,Yb,t),dec)
print(sol, sol - np.cos(t))

1.0 - 0.29110730848387*x
0.8690017 -0.0314454023526769
1.3755593581828*x*(3.0 - 3.33333333333333*x) + 0.69067774252296*x*(3.33333333333333*x - 2.0) + 1.0*(1.0 - 1.66666666666667*x)*(1.0 - 1.11111111111111*x)
-0.43108686701427*x**2 - 0.032455188275307*x + 1.0
0.8981001 -0.00234700235267693


#Cuarta Pregunta

In [104]:
def Hermit(X, Y, D, x = sp.Symbol('x'), dec = 5):
  n = len(X)
  M = np.concatenate((np.transpose([[X],[X]]).reshape((2*n,1)), np.transpose([[Y],[Y]]).reshape((2*n,1))), 1)

  p = Y[0] * 1

  f = np.zeros(2*n)
  for i in range(0,2*n-1,2):
    f[i] = D[i//2]
  for i in range(n-1):
    f[2*i+1] = (Y[i+1] - Y[i])/(X[i+1] - X[i])

  p += np.round(f[0], dec)*(x-M[0,0])
  M = np.append(M, np.transpose([f]), 1)

  for i in range(2,2*n):
    l = []
    for j in range(2*n):
      try:
        l.append((M[j+1,i] - M[j,i])/(M[j+i,0] - M[j,0]))
      except:
        l.append(0)
    M = np.append(M, np.transpose([l]), 1)

    s = 1
    for j in range(i):
      a = x-np.round(M[j,0], dec)
      s *= a
    p += np.round(M[0,i+1], dec)*s

  tx = ['X', 'Y']
  for i in range(1,2*n):
    comas = ','*i
    tx.append(f'f({comas})')

  return pd.DataFrame(np.round(M,dec), columns=tx), p



In [105]:
X = [-1/4, 1/4]
Y = [1.33203, 0.800781]
D = [0.4375, -0.625]
p = x**4 - x**3 + x**2 - x + 1
print(p.subs(x,-0.25), p.subs(x,0.25))
print(sp.diff(p).subs(x,-0.25), sp.diff(p).subs(x,0.25))
# print(sp.diff(p).subs(x,0.734979), sp.diff(p).subs(x,0.25))
t, q = Hermit(X,Y,D, dec = 7)
print(t)
print(q)
print(q.subs(x,0))
print(q.expand())

1.33203125000000 0.800781250000000
-1.75000000000000 -0.625000000000000
      X         Y      f(,)     f(,,)    f(,,,)
0 -0.25  1.332030  0.437500 -2.999996  7.749984
1 -0.25  1.332030 -1.062498  0.874996  0.000000
2  0.25  0.800781 -0.625000  0.000000  0.000000
3  0.25  0.800781  0.000000  0.000000  0.000000
0.4375*x + 7.749984*(x - 0.25)*(x + 0.25)**2 - 2.999996*(x + 0.25)**2 + 1.441405
1.13281175000000
7.749984*x**3 - 1.0625*x**2 - 1.546872*x + 1.13281175


In [106]:
from sympy import symbols, Eq, solve

# Definir las variables
x, y, z = symbols('x y z')

# Definir las ecuaciones
eq1 = Eq(30*x + 20*y + 20*z, 340)
eq2 = Eq(20*x + 30*y + 20*z, 320)
eq3 = Eq(x + y + z, 14)

# Resolver el sistema de ecuaciones
sol = solve((eq1, eq2, eq3), (x, y, z))

# Mostrar la solución
print("Solución:")
print(f"x (Inglaterra) = {sol[x]}")
print(f"y (Francia) = {sol[y]}")
print(f"z (España) = {sol[z]}")


Solución:
x (Inglaterra) = 6
y (Francia) = 4
z (España) = 4


In [107]:
from sympy import symbols, Eq, solve

# Definir las variables
x, y, z = symbols('x y z')

# Definir las ecuaciones
eq1 = Eq(-2*x + y + 6*z, 18)  # -2x + y + 6z = 18
eq2 = Eq(5*x + 8*z, -16)       # 5x + 8z = -16
eq3 = Eq(3*x + 2*y - 10*z, -3) # 3x + 2y - 10z = -3

# Resolver el sistema de ecuaciones
sol = solve((eq1, eq2, eq3), (x, y, z))

# Mostrar la solución
print("Solución:")
print(f"x = {sol[x]}")
print(f"y = {sol[y]}")
print(f"z = {sol[z]}")


Solución:
x = -4
y = 7
z = 1/2
